<a href="https://colab.research.google.com/github/junhyeokk/boomhill24/blob/main/data/kart-data-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [151]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

In [152]:
def mse(img): # Error between input image and white image
  return np.sum((255-img)**2)/(np.shape(img)[0]*np.shape(img)[1])
  
def is_pushed(img): # image is pushed image if error is lager than threshold(200) (check image is almost white)
                    # when not pushed, image is almost white
  return 1 if mse(img) > 200 else 0

In [153]:
# 저장 path 설정
drivepath = "/content/gdrive/MyDrive/"
vidname = "m2"
vidpath = os.path.join(drivepath,vidname+".mp4")
cappath = os.path.join(drivepath,vidname)
csvpath = os.path.join(cappath,vidname+".csv")

if not os.path.exists(cappath):
  os.makedirs(cappath)


In [154]:
# csv, 비디오 파일 open
cap = cv2.VideoCapture(vidpath)
fcsv = open(csvpath,'w',encoding='utf-8')
wrcsv = csv.writer(fcsv)

영상 추출 관련 기본 데이터

In [155]:
fps = round(cap.get(cv2.CAP_PROP_FPS))
t = 30          # Start time
end_sec = 110   # End time
interval = 5    # Time interval
# 단위는 초

print(f"fps : {fps}")

fps : 60


In [156]:
# top left position of key
# up, left, right, ctrl, shift, down 
key_pos = [(1265,914), (1188,992), (1343,992), (1092,992), (1092,914), (1265,992)]

key_size = 20

previnput = "000000"

In [157]:
while t <= end_sec:
  cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
  ret,frame = cap.read()

  if not ret:
    print("영상 시간 초과 또는 영상 관련 오류")

  # 키 입력 추출
  key_array = [is_pushed(frame[index[1]:index[1]+key_size,index[0]:index[0]+key_size]) for index in key_pos]

  # 추출된 키 입력을 string type 변경
  keystring = ''.join(str(key) for key in key_array)
  print(f"key string at {t} is {keystring}")

  # 게임 화면만 저장
  cropped_img = frame[:1080,:1440]

  # 이미지 파일에서 키 뷰어부분 마스킹
  frame[881:1080,1056:1440] = 0

  # 이미지 저장 경로
  imgpath = os.path.join(cappath,f"t{t}.png")

  # 이미지 저장
  cv2.imwrite(imgpath, cropped_img)

  # csv 파일 저장
  wrcsv.writerow([imgpath,previnput])

  t += interval
  previnput = keystring

key string at 30 is 000000
key string at 35 is 000000
key string at 40 is 000000
key string at 45 is 000000
key string at 50 is 110000
key string at 55 is 110000
key string at 60 is 100000
key string at 65 is 110000
key string at 70 is 101000
key string at 75 is 101000
key string at 80 is 100000
key string at 85 is 100000
key string at 90 is 100000
key string at 95 is 101000
key string at 100 is 101000
key string at 105 is 101000
key string at 110 is 101010


In [158]:
cap.release()
fcsv.close()